In [1]:
# https://platform.olimpiada-ai.ro/problems/58

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

In [3]:
train = pd.read_csv('/kaggle/input/spam-email/train.csv')
test = pd.read_csv('/kaggle/input/spam-email/test.csv')

train.shape, test.shape

((66758, 3), (16690, 2))

In [4]:
train.head(3)

,sample_id,label,text
0,6031,0,mumia w wrote on escapenumber escapenumber esc...
1,75167,1,oh my dear sir her throat is so much better t...
2,69801,1,many more are victims of other cons telemarket...


In [19]:
from sklearn.model_selection import train_test_split
from catboost import Pool

features = ['text']
text_features = ['text']
target_col = ['label']

X, y = train[features], train[target_col]
X_test = test[features]

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

train_pool = Pool(X_train, y_train, text_features=text_features)
valid_pool = Pool(X_valid, y_valid, text_features=text_features)
full_pool = Pool(X, y, text_features=text_features)

In [21]:
from catboost import CatBoostClassifier

params = {
    'iterations': 300,
    'loss_function': 'Logloss',
    'eval_metric': 'AUC',
    'metric_period': 25,
    'max_depth': 6
}

model = CatBoostClassifier(**params)

model.fit(train_pool, eval_set=valid_pool)

Learning rate set to 0.142914
0:	test: 0.9713755	best: 0.9713755 (0)	total: 167ms	remaining: 49.8s
25:	test: 0.9970065	best: 0.9970065 (25)	total: 3.93s	remaining: 41.4s
50:	test: 0.9976263	best: 0.9976263 (50)	total: 7.49s	remaining: 36.6s
75:	test: 0.9979513	best: 0.9979513 (75)	total: 11.1s	remaining: 32.7s
100:	test: 0.9981419	best: 0.9981419 (100)	total: 14.6s	remaining: 28.8s
125:	test: 0.9982878	best: 0.9982878 (125)	total: 18.1s	remaining: 25.1s
150:	test: 0.9983883	best: 0.9983883 (150)	total: 21.8s	remaining: 21.5s
175:	test: 0.9984664	best: 0.9984664 (175)	total: 25.4s	remaining: 17.9s
200:	test: 0.9985088	best: 0.9985088 (200)	total: 29s	remaining: 14.3s
225:	test: 0.9985617	best: 0.9985617 (225)	total: 32.5s	remaining: 10.7s
250:	test: 0.9985861	best: 0.9985861 (250)	total: 36.1s	remaining: 7.04s
275:	test: 0.9986357	best: 0.9986357 (275)	total: 39.7s	remaining: 3.45s
299:	test: 0.9986647	best: 0.9986647 (299)	total: 43.2s	remaining: 0us

bestTest = 0.9986646886
bestIterat

In [22]:
from sklearn.metrics import roc_auc_score

y_pred = model.predict_proba(X_valid)[:, 1]

score = roc_auc_score(y_valid, y_pred)

print(f'Score: {score:.5f}')

Score: 0.99866


In [23]:
model = CatBoostClassifier(**params)

model.fit(full_pool)

Learning rate set to 0.186843
0:	total: 173ms	remaining: 51.9s
25:	total: 4.36s	remaining: 46s
50:	total: 8.21s	remaining: 40.1s
75:	total: 12.2s	remaining: 35.8s
100:	total: 16.1s	remaining: 31.7s
125:	total: 20s	remaining: 27.6s
150:	total: 23.9s	remaining: 23.6s
175:	total: 27.7s	remaining: 19.5s
200:	total: 31.5s	remaining: 15.5s
225:	total: 35.3s	remaining: 11.6s
250:	total: 39.1s	remaining: 7.64s
275:	total: 42.8s	remaining: 3.72s
299:	total: 46.5s	remaining: 0us


In [25]:
y_pred = model.predict_proba(X_test)[:, 1]

sids, dpids, answers = [], [], []

for i, row in test.iterrows():
    length = len(row['text'])
    cnt = row['text'].count('free')

    sids.append(1)
    dpids.append(row['sample_id'])
    answers.append(length)

    sids.append(2)
    dpids.append(row['sample_id'])
    answers.append(cnt)

    sids.append(3)
    dpids.append(row['sample_id'])
    answers.append(y_pred[i])

subm = pd.DataFrame({
    'subtaskID': sids,
    'datapointID': dpids,
    'answer': answers
})

subm.to_csv("submission.csv", index=False)

subm.head()

,subtaskID,datapointID,answer
0,1,75707,237.000000
1,2,75707,0.000000
2,3,75707,0.999431
3,1,4026,946.000000
4,2,4026,0.000000
